In [ ]:
import os
import random
from collections import Counter

In [ ]:
def open_file(filename, mode='r'):
    return open(filename, mode=mode, encoding='utf-8', errors='ignore')


def read_vocab(vocab_path):
    words = open_file(vocab_path).read().strip().split('\n')
    word_to_id = dict(zip(words, range(len(words))))
    return words, word_to_id


def build_vocab(data_path, vocab_path, vocab_size):
    tokens = ['<sos>', '<eos>', '<unk>']  # 词汇表中的几个重要标记
    all_words = open_file(data_path).read().strip().replace('\n', ' ').split()
    count_pairs = Counter(all_words).most_common(vocab_size)
    words, _ = list(zip(*count_pairs))
    tokens += words
    open_file(vocab_path, 'w').write('\n'.join(tokens) + '\n')


def text_to_id(text, w2id, unk_token):
    return [w2id[x] if x in w2id else unk_token for x in text.split()]


def id_to_text(ids, words):
    return ' '.join([words[x] for x in ids])

In [ ]:
class Corpus(object):
    def __init__(self, data_path, vocab_path, vocab_size=10000):
        assert os.path.exists(data_path)

        if not os.path.exists(vocab_path):
            build_vocab(data_path, vocab_path, vocab_size - 3)

        self.words, self.word_to_id = read_vocab(vocab_path)

        self.tokenize(data_path)

    def tokenize(self, data_path):
        eos_token = self.word_to_id['<eos>']
        unk_token = self.word_to_id['<unk>']
        lines = []
        data = []
        for line in open_file(data_path):
            if len(line.strip()) == 0:
                data.extend(list(zip(lines[:-1], lines[1:])))
                lines = []
            line_ids = text_to_id(line + ' <eos>', self.word_to_id, unk_token)
            if line_ids.count(unk_token) < len(line_ids) * 0.2:
                lines.append(line_ids)

        # 打乱，分离数据集
        random.shuffle(data)
        data_len = len(data)
        self.data_train = data[:int(0.7 * data_len)]
        self.data_val = data[int(0.7 * data_len):int(0.8 * data_len)]
        self.data_test = data[int(0.8 * data_len):]

    def __repr__(self):
        return "Vocab size: {}\nTrain len: {}\nValidation len: {}\nTest len: {}".format(len(self.words),
                                                                                        len(self.data_train),
                                                                                        len(self.data_val),
                                                                                        len(self.data_test))

In [ ]:
corpus = Corpus('lyric_full.txt', 'lyric_vocab.txt')
print(corpus)

In [ ]:
r_t = random.choice(corpus.data_train)
print(id_to_text(r_t[0], corpus.words))
print(id_to_text(r_t[1], corpus.words))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, embedding, hidden_size, n_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding = embedding
        self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers, dropout=dropout)
        
    def forward(self, input_s, hidden=None):
        embedded = self.embedding(input_s).view(len(input_s), 1, -1)
        outputs, hidden = self.rnn(embedded, hidden)
        return outputs, hidden

In [ ]:
embedding_size = 500
hidden_size = 500
n_layers = 2

vocab_size = len(corpus.words)
embedding = nn.Embedding(vocab_size, embedding_size)
encoder_test = EncoderRNN(embedding, hidden_size, n_layers)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(DecoderRNN, self).__init__()

        # Keep for reference
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers, dropout=dropout)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input_s, last_hidden):
        # Note: we run this one step at a time
        # Get the embedding of the current input word (last output word)
        embedded = self.embedding(input_s)
        embedded = self.embedding_dropout(embedded).view(1, 1, -1)

        # Get current hidden state from input word and last hidden state
        rnn_output, hidden = self.rnn(embedded, last_hidden)

        rnn_output = rnn_output.squeeze(0) # S=1 x B x N -> B x N

        # Finally predict next token (Luong eq. 6, without softmax)
        output = self.out(rnn_output)

        # Return final output, hidden state, and attention weights (for visualization)
        return output, hidden

In [ ]:
decoder_test = DecoderRNN(embedding, hidden_size, vocab_size, n_layers)
decoder_test

In [ ]:
def variables_from_pair(pair):
    input_var = Variable(torch.LongTensor(pair[0]))
    target_var = Variable(torch.LongTensor(pair[1]))
    if use_cuda:
        input_var = input_var.cuda()
        target_var = target_var.cuda()
    return (input_var, target_var)

In [ ]:
pair = random.choice(corpus.data_train)
input_var, target_var = variables_from_pair(pair)
print(input_var.size(), target_var.size())
encoder_outputs, encoder_hidden = encoder_test(input_var)
print(encoder_outputs.size())

decoder_hidden = encoder_hidden
for i in range(len(target_var)):
    decoder_output, decoder_hidden = decoder_test(target_var[i], decoder_hidden)
    print(decoder_output.size(), decoder_hidden[0].size())

In [ ]:
def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0

    target_length = target_variable.size(0)
    encoder_outputs, encoder_hidden = encoder(input_variable)

    decoder_input = Variable(torch.LongTensor([[corpus.word_to_id['<sos>']]]))
    decoder_hidden = encoder_hidden
    if use_cuda:
        decoder_input = decoder_input.cuda()

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        loss += criterion(decoder_output, target_variable[di])
        decoder_input = target_variable[di] 

    # Backpropagation
    loss.backward()
    torch.nn.utils.clip_grad_norm(encoder.parameters(), clip)
    torch.nn.utils.clip_grad_norm(decoder.parameters(), clip)
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0] / target_length

In [ ]:
learning_rate = 0.0001
clip = 5.0
decoder_learning_ratio = 5.0
encoder_optimizer = optim.Adam(encoder_test.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder_test.parameters(), lr=learning_rate * decoder_learning_ratio)

criterion = nn.CrossEntropyLoss(size_average=False)

In [ ]:
loss = train(input_var, target_var, encoder_test, decoder_test, encoder_optimizer, decoder_optimizer, criterion)
loss

In [ ]:
print_per_epoch = 10
epochs = 50000
total_loss = 0
for i in range(1, epochs):
    input_var, target_var = variables_from_pair(random.choice(corpus.data_train))
    total_loss += train(input_var, target_var, encoder_test, decoder_test, encoder_optimizer, decoder_optimizer, criterion)
    if i % print_per_epoch == 0:
        print(i, total_loss / print_per_epoch)
        total_loss = 0